In [129]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic

ROOT = '/home/robert/.config/JetBrains/DataSpell2021.3/projects/MLS'

In [130]:
AllYears = pd.read_csv(f'{ROOT}/Results/AllYears.csv', index_col=0)
AllYears.drop(['Venue'], axis=1, inplace=True)
AllYears = AllYears.sort_values(by=['Date'])

In [131]:
teams = {'Atlanta Utd': ['ATL', 'Atlanta'],
         'Austin FC': ['ATX', 'Austin'],
         'CF Montréal': ['MTL', 'Montreal'],
         'Charlotte FC': ['CLT', 'Charlotte'],
         'Chicago Fire': ['CHI', 'Chicago'],
         'Chivas USA': ['CHV', 'Los Angeles'],
         'Colorado Rapids': ['COL', 'Denver'],
         'Columbus Crew': ['CLB', 'Columbus'],
         'D.C. United': ['DC', 'Washington DC'],
         'Dallas Burn': ['DAL', 'Dallas'],
         'FC Cincinnati': ['CIN', 'Cincinnati'],
         'FC Dallas': ['DAL', 'Dallas'],
         'Houston Dynamo': ['HOU', 'Houston'],
         'Inter Miami': ['MIA', 'Miami'],
         'KC Wiz': ['SKC', 'Kansas City'],
         'KC Wizards': ['SKC', 'Kansas City'],
         'LA Galaxy': ['LA', 'Los Angeles'],
         'Los Angeles FC': ['LAFC', 'Los Angeles'],
         'MetroStars': ['RBNY', 'New York'],
         'Miami Fusion': ['MIAF', 'Miami'],
         'Minnesota Utd': ['MIN', 'Saint Paul'],
         'Montreal Impact': ['MTL', 'Montreal Impact'],
         'NY Red Bulls': ['RBNY', 'New York'],
         'NYCFC': ['NYC', 'New York'],
         'Nashville': ['NSH', 'Nashville'],
         'New England': ['NE', 'Foxborough'],
         'Orlando City': ['ORL', 'Orlando'],
         'Philadelphia': ['PHI', 'Chester'],
         'Portland Timbers': ['POR', 'Portland'],
         'Real Salt Lake': ['RSL', 'Sandy'],
         'San Jose': ['SJ', 'San Jose'],
         'Seattle': ['SEA', 'Seattle'],
         'Sporting KC': ['SKC', 'Kansas City'],
         'Tampa Bay': ['TB', 'Tampa Bay'],
         'Toronto FC': ['TOR', 'Toronto'],
         'Vancouver': ['VAN', 'Vancouver']
             }

In [132]:
def get_distance(city1, city2):
    def get_lat_long(city):
        lat = gpd.tools.geocode(city).geometry.y.values[0]
        long = gpd.tools.geocode(city).geometry.x.values[0]
        return (lat, long)
    return int(geodesic(get_lat_long(city1), get_lat_long(city2)).mi)

In [133]:
for i in AllYears.index:
    AllYears.at[i, 'Home'] = teams[AllYears.at[i, 'Home']][0]
    AllYears.at[i, 'Away'] = teams[AllYears.at[i, 'Away']][0]
    if AllYears.at[i, 'Winner'] == AllYears.at[i, 'Winner']:
        AllYears.at[i, 'Winner'] = teams[AllYears.at[i, 'Winner']][0]
    else:
        AllYears.at[i, 'Winner'] = 'Draw'

In [134]:
def get_regular_season(team_code):
    home_games = AllYears[AllYears.Home==team_code][AllYears[AllYears.Home==team_code].Round=='Regular Season']
    away_games = AllYears[AllYears.Away==team_code][AllYears[AllYears.Away==team_code].Round=='Regular Season']
    return pd.concat([home_games, away_games]).sort_values(by=['Date'])

In [146]:
def get_overall_record(team_code):
    df = get_regular_season(team_code)
    wins = len(df[df.Winner==team_code])
    draws = len(df[df.Winner=='Draw'])
    losses = len(df) - wins - draws
    return wins, draws, losses
get_overall_record('SKC')

(337, 179, 302)